# COIN Project

Authors:

    - Mona
    - Roman
    - Nick
    - Mateo

# Setup

## Packages

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
import dotenv
import os
import yaml
import datetime
from pprint import pprint

## Environment variables

In [2]:
dotenv.load_dotenv()

True

In [3]:
with open("../twitter.yml", "r") as f:
    config = yaml.safe_load(f)

## Database connection

In [4]:
username = os.environ["DB_USERNAME"]
password = os.environ["DB_PASSWORD"]
host = os.environ["DB_HOST"]
port = os.environ["DB_PORT"]
name = os.environ["DB_NAME"]

In [5]:
engine = sa.create_engine("mssql+pymssql://{}:{}@{}/{}".format(username, password, host, name))

## Tweepy

In [6]:
client = tweepy.Client(bearer_token=os.environ["BEARER_TOKEN"], wait_on_rate_limit=True)

In [ ]:
auth = tweepy.OAuth2BearerHandler(bearer_token=os.environ["BEARER_TOKEN"])
api = tweepy.API(auth)

# Twitter API

## Get UserIDs from Twitter

In [7]:
users = {user: client.get_user(username=user).data.id for user in config["accounts"]}

In [8]:
users

{'Fridays4future': 1053768884732547072,
 'FridayForFuture': 1072187272815149057,
 'GretaThunberg': 1006419421244678144,
 'luisaneubauer': 2689488949}

In [9]:
user_id = users["FridayForFuture"]

## Get tweets from user

In [132]:
expansions = ["author_id", "place_id"]
tweet_fields = ["created_at", "geo", "public_metrics", "text", "context_annotations", "entities"]

In [ ]:
response = tweepy.Paginator(
    client.get_users_tweets,
    id=user_id,
    start_time=datetime.datetime(2021, 1, 1),
    end_time=datetime.datetime.now(),
    expansions=expansions,
    tweet_fields=tweet_fields
).flatten(100)

In [ ]:
response = list(response)

In [113]:
tweets = [
    {response   "id": r.id,
        "author_id": r.author_id,
        "created_at": r.created_at,
        "geo": r.geo,
        "retweet_count": r.public_metrics["retweet_count"],
        "reply_count": r.public_metrics["reply_count"],
        "like_count": r.public_metrics["like_count"],
        "qoute_count": r.public_metrics["quote_count"],
        "text": r.text,
        "entities": str(r.entities),
        "context_annotations": str(r.context_annotations)
    } for r in response
]

In [120]:
tweets = tweets.copy()

In [121]:
df = pd.DataFrame(tweets)

In [122]:
df.head(10)

,id,author_id,created_at,geo,retweet_count,reply_count,like_count,qoute_count,text,entities,context_annotations
0,1521569732289310720,1072187272815149057,2022-05-03 19:17:48+00:00,None,54,11,311,1,Während in der NRW-#Wahlarena ernsthaft noch P...,"{'hashtags': [{'start': 19, 'end': 29, 'tag': ...",[]
1,1521533572926038017,1072187272815149057,2022-05-03 16:54:07+00:00,None,28,38,182,3,Gestern wurde in Mannheim ein Mann von der Pol...,"{'hashtags': [{'start': 136, 'end': 150, 'tag'...",[]
2,1521399841514999809,1072187272815149057,2022-05-03 08:02:43+00:00,None,76,33,299,9,Während wir eigentlich weg von fossilen Rohsto...,"{'urls': [{'start': 254, 'end': 277, 'url': 'h...","[{'domain': {'id': '10', 'name': 'Person', 'de..."
3,1520843928450355201,1072187272815149057,2022-05-01 19:13:43+00:00,None,28,18,233,1,Für Klimagerechtigkeit braucht es soziale Gere...,"{'urls': [{'start': 193, 'end': 216, 'url': 'h...",[]
4,1520321012138745856,1072187272815149057,2022-04-30 08:35:50+00:00,None,91,17,492,5,Klimaschutz ist Gesundheitsschutz! https://t.c...,"{'urls': [{'start': 35, 'end': 58, 'url': 'htt...",[]
5,1520114614062747655,1072187272815149057,2022-04-29 18:55:41+00:00,None,59,26,321,5,"Solidarität mit der Ukraine bedeutet, dass die...","{'hashtags': [{'start': 178, 'end': 194, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New..."
6,1520036287390048264,1072187272815149057,2022-04-29 13:44:26+00:00,None,84,36,312,5,Seit Kriegsbeginn hat 🇩🇪 über 9 Milliarden Eur...,"{'hashtags': [{'start': 195, 'end': 211, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New..."
7,1520032148048252930,1072187272815149057,2022-04-29 13:27:59+00:00,None,36,14,254,3,Angesichts des grausamen Kriegs stehen wir in ...,"{'hashtags': [{'start': 148, 'end': 164, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New..."
8,1520028899253530626,1072187272815149057,2022-04-29 13:15:05+00:00,None,23,17,209,1,Bundesregierung:\n“Unsere Sanktionen treffen P...,"{'hashtags': [{'start': 192, 'end': 208, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New..."
9,1520027479334957058,1072187272815149057,2022-04-29 13:09:26+00:00,None,22,14,206,0,Jahrelang haben @ManuelaSchwesig &amp; weitere...,"{'hashtags': [{'start': 219, 'end': 235, 'tag'...",[]


In [116]:
df.shape

(1601, 11)

In [119]:
df.dtypes

id                                   int64
author_id                            int64
created_at             datetime64[ns, UTC]
geo                                 object
retweet_count                        int64
reply_count                          int64
like_count                           int64
qoute_count                          int64
text                                object
entities                            object
context_annotations                 object
dtype: object

In [129]:
df["entities"] = df["entities"].astype(str)
df["created_at"] = df["created_at"].astype(str)
df["context_annotations"] = df["context_annotations"].astype(str)

In [131]:
df.to_csv("tweets.csv", index=False)

In [130]:
df.to_sql("tweets", con=engine, index=False, if_exists="replace")

ValueError: expected a simple type, a tuple or a list